# Small consumer 2-zone test

Using generated price timeseres, assume that national grid system is beyond consumer's control.

What we then need to do is to introduce a second zone (or “bus”) representing consumer’s assets

1. Establish a second “zone” with a single connector between it and the first zone.
2. The second zone should contain consumer’s:
   a. demand for power ... using `outputs_plan_national-grid/???.csv`, rescale National Grid mean & std. to 1%, and use to generate a random timeseries of sorm form
   b. diesel generators ... (for now) set the *capacity* of these generators to be effectively infinite, but the consumption cost to be *very high* (say 2-3 times that of the peaking plant in the national grid zone but not as high as unmet demand).  
3. The connector between the “zones” should be:
   a. Effectively infinite in capacity when taking power from the national grid zone to the consumer zone
   b. Zero cost per unit of power transferred
   c. Unidirectional (only takes power from Nat Grid to consumer, not vice versa). see https://calliope.readthedocs.io/en/stable/user/advanced_constraints.html#one-way-transmission-links

Big picture: larger zone (National Grid) sets its own prices based on national demand/wind etc. The smaller zone is always able to meet it’s own internal demand if it chooses to (diesel capacity is very large) but would usually *prefer* to take power from national grid rather than use it’s own diesel generators. No power export from consumer to grid.

Further work:

* Shrink national grid generation capacities (introduce shortages), observe impact on consumer zone.
* Look at swapping diesel generators to a *storage* generator(s).

In [ ]:
# Suppress minor warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import calliope
import models
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Read in National grid planning output (full 1980-2017)

From timeseries calculate mean and standard deviation of demand.
Change back from negative convention (calliope) to positive demand convention (readable)

In [ ]:
# read csv
df = pd.read_csv('outputs_plan_national-grid/inputs_resource.csv',
                 usecols=['timesteps', 'resource', 'techs'], index_col='timesteps')
# split demand / wind /solar into 3 separate columns
national_demand_index = pd.to_datetime(df[df['techs'] == 'demand_power'].index)
national_demand = pd.DataFrame(dict(), index=national_demand_index)
national_demand['demand'] = - df[df['techs'] == 'demand_power']['resource']
national_demand['wind'] = df[df['techs'] == 'wind']['resource']
national_demand['solar'] = df[df['techs'] == 'solar']['resource']
del df

In [ ]:
national_demand_stats = mean, std = [stat(national_demand.demand) for stat in [np.mean, np.std]]

### Generate a random timeseries 
form with similar overall mean/stddev to UK wide demand (scaled to 1%), ensure no negative values

In [ ]:
# ensure always using the same seed (which we obtained one-off using RNG)
np.random.seed(285183912)

# normal distribution parameters = 1% National Grid mean/std dev.
normal_dist = [stat*0.01 for stat in national_demand_stats]

# Sample from normal distribution
demand_region2 = np.random.normal(*normal_dist, len(national_demand))

# Force positives (very minute chance of this occuring...)
demand_region2[demand_region2 < 0] = 0

In [ ]:
demand_region2

### Load secondary zone demand into a DataFrame
Allows easy loading into ts_data

In [ ]:
df_region2 = pd.DataFrame({'demand_region2': demand_region2},
                          index=pd.to_datetime(national_demand.index))

# `operational` Calliope 2-zone model (with infinite diesel cap)

Note: can shrink operational range to subset, eg. 2017 year only

In [ ]:
date_start, date_end = '2015', '2017'

In [ ]:
# Import timeseries data demand / wind (as in 1_region)
ts_data = models.load_time_series_data('2_region', additional_data=df_region2)

# Crop to date range
ts_data = ts_data.loc[date_start:date_end]

display(ts_data.head(6))

In [ ]:
# Read in generation capacities
generation_capacities_planned = pd.read_csv('outputs_plan_national-grid/results_energy_cap.csv')

# Rename techs
generation_capacities = dict()
for tech, cap in zip(generation_capacities_planned['techs'],
                     generation_capacities_planned['energy_cap']):
    if tech not in ['unmet', 'demand_power']:
        key = f'cap_{tech}_region1'
        generation_capacities[key] = cap

# Insert diesel generators, transmission capacity = max possible region2 demand
generation_capacities['cap_generators_region2'] = max(ts_data['demand_region2'])
generation_capacities['cap_transmission_region1_region2'] = max(ts_data['demand_region2'])


# Display
display(generation_capacities)

In [ ]:
# Create the model with fixed capacities
model = models.TwoRegionModel(ts_data, 'operate', fixed_caps=generation_capacities)
display(model.preview('inputs.resource'))

In [ ]:
model.run()
model.get_summary_outputs()

In [ ]:
# Export all model outputs to CSV (creates directory called 'outputs_operate')
output_folder = 'outputs_operational_two-zone'
models.rm_folder(output_folder)
model.to_csv(output_folder)

In [ ]:
# Generate HTML plots
for var in ['power', 'cost_var', 'resource']:
    plot_html = model.plot.timeseries(array=var, html_only=True)
    models.save_html(plot_html, f'plots/operate_2_{var}.html', f'{var} plot')

In [ ]:
model.preview('results.systemwide_capacity_factor', loc='carriers', time_idx=False,
              index=model.results.systemwide_capacity_factor.techs.values)

In [ ]:
for cost, tech in zip(model.inputs.cost_om_con.values[0],
                      model.inputs.loc_techs_om_cost.values):
    print(cost, tech)

### Plot carrier production

Manually plotting with matplotlib (below) disagrees with calliope plotting routine output! Why?

Here we (correctly!) see transmission in region 2 as the power source, not generators

In [ ]:
# Time slice
s = slice(100, 500)

# Setup plot
fig, axs = plt.subplots(2)

# Cumulative array starts at zero
cumulative = [np.zeros_like(model.results.carrier_prod.values[0]), np.zeros_like(model.results.carrier_prod.values[0])]

# Plot cumulative production (ie. wind, wind + baseload, etc.)
for tech_label, res in zip(model.results.carrier_prod.loc_tech_carriers_prod.values,
                     model.results.carrier_prod.values):
    region, tech, _ = tech_label.split('::')
    reg_idx = {'region1': 0, 'region2': 1}[region]
    axs[reg_idx].fill_between(model.inputs.timesteps[s], cumulative[reg_idx][s], (cumulative[reg_idx] + res)[s], label=tech)
    cumulative[reg_idx] += res
    
# Legend, labels and formatting
for ax in axs:
    ax.legend(bbox_to_anchor=(1,1), loc="upper left")
    ax.set_xlabel('date')
    ax.set_ylabel('Carrier Production')
fig.autofmt_xdate()